# Generate the figures for the manuscript

In [ ]:
import platform
import os
import glob
import pandas
import dask
from dask.distributed import Client, LocalCluster
import dask_image.imread
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
import seaborn
import numpy
from tqdm.auto import tqdm, trange
import imageio
import skimage

In [ ]:
# Import our own parsing functions which we've added as submodule
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [ ]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

In [ ]:
# Start cluster and client now, after setting tempdir
try:
    cluster = LocalCluster()
except PermissionError:
    print('Mount the Fast_SSD, otherwise we cannot use it for saving the temporary files!')
    print('Then rerun this cell.')
client = Client(cluster)

In [ ]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

In [ ]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [ ]:
# Set seaborn context
# context: one of {paper, notebook, talk, poster}
seaborn.set_context('paper')

In [ ]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
#plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 300
# Since we fiddle withthe figure size below, keep the default one handy
figsize_default = plt.rcParams['figure.figsize']

In [ ]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [ ]:
# Different locations if running either on Linux or Windows
FastSSD = True
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', 'research-storage-djonov')
elif 'Darwin' in platform.system():
    # First mount smb://resstore.unibe.ch/ana_rs_djonov/data in the Finder
    FastSSD = False
    BasePath = os.path.join('/Volumes/data/')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('V:\\')
        else:
            BasePath = os.path.join('V:\\')
Root = os.path.join(BasePath, 'Aaldijk', 'PelvicFloor')
print('We are loading all the data from %s' % Root)

In [ ]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [ ]:
# Make directory for output
OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
print('We are saving all the output to %s' % OutPutDir)
os.makedirs(OutPutDir, exist_ok=True)

In [ ]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [ ]:
# Get *all* log files, unsorted but faster than with glob
print('Searching for all log files in %s' % Root)
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

In [ ]:
# Drop all the scans that are not of Foetus02
for c, row in Data.iterrows():
    if 'Foetus02' not in row.LogFile:
        Data.drop([c], inplace=True)
Data.reset_index(inplace=True)

In [ ]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [ ]:
# Get rid of all logfiles that we don't want and need
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:  # drop all non-rec folders
        Data.drop([c], inplace=True)
    elif 'SubScan' in row.Folder:  # drop all partial reconstructions which might be there from synchronization
        Data.drop([c], inplace=True)        
    elif 'rectmp.log' in row.LogFile:  # drop all temporary logfiles
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [ ]:
# Generate us some meaningful colums
Data['Sample'] = [l[len(Root)+1:].split(os.sep)[0] for l in Data['LogFile']]
Data['SampleName'] = [sn.split('_')[0] for sn in Data['Sample']]
Data['Scan'] = ['_'.join(l[len(Root)+1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [ ]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [ ]:
Data.SampleName.unique()

In [ ]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
# `find . -name "*rec*.png" -type f -mtime +333 -delete`
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
Data = Data[Data['Number of reconstructions'] > 0]
Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

In [ ]:
# Get scanning parameters to doublecheck from logfiles
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Voxelsize'] = [pixelsize(log, rounded=True) for log in Data['LogFile']]
Data['CameraWindow'] = [projection_size(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposuretime(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]
Data['Scan time'] = [duration(log) for log in Data['LogFile']]

In [ ]:
# Sort our dataframe by scan date
Data.sort_values(by='Scan date', inplace=True, ignore_index=True)

In [ ]:
# Get reconstruction parameters to doublecheck from logfiles
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]
Data['ROI'] = [region_of_interest(log) for log in Data['LogFile']]
Data['Rot'] = [crosssection_rotation(l) for l in Data.LogFile]

In [ ]:
# Calculate time 'spent' since start
Data['Time passed'] = [sd - Data['Scan date'].min() for sd in Data['Scan date']]
# Also extract days, rounded
Data['Days passed'] = [t.round('d') for t in Data['Time passed']]

In [ ]:
# Load all reconstructions into ephemereal DASK arrays
#Reconstructions = [None] * len(Data)
#for c, row in tqdm(Data.iterrows(),
#                   desc='Load reconstructions',
#                   total=len(Data)):
#    Reconstructions[c] = dask_image.imread.imread(os.path.join(row['Folder'],
#                                                               '*rec*.png'))

In [ ]:
# Convert all reconstructions into zarrays on disk, optimized for fast reading
# Partially based on http://stackoverflow.com/a/39195332/323100
# and on /LungMetastasis/HighResolutionScanAnalysis.ipynb
Data['OutputNameRec'] = [((os.path.join(os.path.dirname(f), s)) + '.' + rf + '.zarr') for f, s, rf in zip(Data['Folder'],
                                                                                                          Data['Sample'],
                                                                                                          Data['Scan'])]
for c, row in tqdm(Data.iterrows(),
                            desc='Converting reconstructions to .zarr',
                            total=len(Data)):
    if not os.path.exists(row['OutputNameRec']):
        print('%2s/%2s: Reading %s reconstructions and saving to %s' % (c + 1,
                                                                        len(Data),
                                                                        row['Number of reconstructions'],
                                                                        row['OutputNameRec'][len(Root) + 1:]))
        Reconstructions = dask_image.imread.imread(os.path.join(row['Folder'], '*rec*.png'))[:,:,:,0]
        # Save out to automatically rechunked and un-compressed zarr files.
        # This takes a *long* time, but subsequent operations are much quicker afterwards
        Reconstructions.rechunk('auto').to_zarr(row['OutputNameRec'],
                                                overwrite=True)

In [ ]:
# Load the reconstructions from the zarrays
Reconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRec']]

In [ ]:
Reconstructions[0]

In [ ]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [ ]:
# # Calculate the histograms of *all* the reconstructions
# # Caveat: dask.da.histogram returns histogram AND bins, making each histogram a 'nested' list of [h, b]
# Data['Histogram'] = [dask.array.histogram(dask.array.array(mip),
#                                           bins=2**8,
#                                           range=[0, 2**8]) for mip in Reconstructions]
# # Actually compute the data and put only h into the dataframe, so we can use it below.
# # Discard the bins
# Data['Histogram'] = [h.compute() for h, b in Data['Histogram']]

In [ ]:
subsample = 2

In [ ]:
# Calculate the Otsu threshold of *all* the reconstructions, subsampled for speed reasons
Data['Treshold_Otsu'] = [skimage.filters.threshold_otsu(rec[::subsample,::subsample,::subsample].compute()) for rec in Reconstructions]

In [ ]:
Data['Treshold_Otsu']

In [ ]:
# # Show *all* the gray value histograms
# # Show *all* the Thresholds, too
# for c, row in sorted(Data.iterrows()):
#     plt.semilogy(row.Histogram,
#                  label='%s' % row.Sample)
#     plt.axvline(row.Treshold_Otsu)
# plt.xlim([0, 255])
# plt.legend()
# plt.show()

In [ ]:
# Mask all the background
# This is *extremely* simple segmentation, but gives us a better estimate on the gray value *in* the sample by discarding the background
Reconstructions_masked = [dask.array.ma.masked_less(rec, threshold) for rec, threshold in zip(Reconstructions,
                                                                                              Data['Treshold_Otsu'])]

In [ ]:
Rec = 12
Slice = 333
plt.subplot(121)
plt.imshow(Reconstructions[Rec][999])
plt.subplot(122)
plt.imshow(Reconstructions_masked[Rec][999])
plt.show()

In [ ]:
# Calculate mean brightness of the reconstructions
# Subsample for speed reasons, the subsample value is given above
subsample = 1  # For the finalized version of the figure we do *not* subsample
Data['MeanBrightness'] = [rec[::subsample,::subsample,::subsample].mean().compute()
                          for rec in Reconstructions]

In [ ]:
# Calculate mean brightness of the masked reconstructions
Data['MeanBrightness_masked'] = [rec[::subsample,::subsample,::subsample].mean().compute()
                                 for rec in Reconstructions_masked]

In [ ]:
Data[['Sample', 'MeanBrightness', 'MeanBrightness_masked']]

In [ ]:
seaborn.scatterplot(data=Data,
                    x='Sample',
                    y='MeanBrightness',
                    label='mean Brightnes')
seaborn.scatterplot(data=Data,
                    x='Sample',
                    y='MeanBrightness_masked',
                    label='mean Brightnes, masked')
plt.legend()
plt.show()

In [ ]:
Data[['Sample', 'Scan', 'Size', 'MeanBrightness',
      'ROI', 'Days passed', 
      'Grayvalue', 'RingartefactCorrection', 'BeamHardeningCorrection', 'DefectPixelMasking', 'Rot']]

In [ ]:
Data[['Sample', 'Scan', 'Size', 'MeanBrightness',
      'ROI', 'Days passed', 
      'Grayvalue', 'RingartefactCorrection', 'BeamHardeningCorrection', 'DefectPixelMasking', 'Rot']].to_csv(os.path.join('data',
                                                                                                                          'ScanDetails.Figure01.csv'))

In [ ]:
Data[['Sample', 'Scan', 'Size', 'MeanBrightness',
      'ROI', 'Days passed', 
      'Grayvalue', 'RingartefactCorrection', 'BeamHardeningCorrection', 'DefectPixelMasking', 'Rot']].to_excel(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Data',
                                                                                                                            'ScanDetails.Figure01.xlsx'))

Define us some helper functions for generating the figures.

In [ ]:
# Adapted from AcinarSize_Johannes/MicroscopyFigure.ipynb
def label_image(image, labeltext, x=None, y=None, color='white', boxcolor=None):
    '''We have to print a label over the image several times'''
    # If we didn't set coordinates, then use defaults
    if not x:
        x=numpy.shape(image)[1] * 0.0309
    if not y:
        y=numpy.shape(image)[0] - (numpy.shape(image)[0] * 0.0309)
    t = plt.gca().text(x,
                       y,
                       labeltext,
                       color=color,
                       fontsize=12,
                       verticalalignment='bottom',
                       horizontalalignment='left')
    if boxcolor is not None:
        t.set_bbox(dict(facecolor=boxcolor,
                        edgecolor=boxcolor,
                        alpha=0.618))
    return

In [ ]:
def extract_directional_slice(stack, direction, coordinates, verbose=False):
    if verbose:
        print('The input stack has a size of %s x %s x %s px' % (stack.shape[0],
                                                                 stack.shape[1],
                                                                 stack.shape[2]))
    if direction == 0:       
        if verbose:
            print('Extracting YZ slice %s of %s' % (coordinates[0], stack.shape[2]))
            print('Marking y=%s' % coordinates[1])
            print('Marking z=%s' % coordinates[2])                   
        # Flip the extracted YZ slice, to be consistent with the OrthoViewer of ImageJ
        extractedslice = dask.array.fliplr(stack[:,:,coordinates[0]])
    elif direction == 1:
        if verbose:
            print('Extracting XZ slice %s of %s' % (coordinates[1], stack.shape[1]))
            print('Marking x=%s' % coordinates[0])
            print('Marking z=%s' % coordinates[2])            
        extractedslice = stack[:,coordinates[1],:]
    elif direction == 2:
        if verbose:
            print('Extracting Z slice %s of %s' % (coordinates[2], stack.shape[0]))
            print('Marking x=%s' % coordinates[0])
            print('Marking y=%s' % coordinates[1])
        extractedslice = stack[coordinates[2],:,:]
    if verbose:
        plt.imshow(extractedslice)
        if direction == 0:
            plt.axhline(coordinates[2], c=seaborn.color_palette()[1])
            plt.axvline(coordinates[1], c=seaborn.color_palette()[0])
            plt.title('YZ (dir %s): coords: %s' % (direction, coordinates))            
        elif direction == 1:
            plt.axhline(coordinates[2], c=seaborn.color_palette()[2])
            plt.axvline(coordinates[0], c=seaborn.color_palette()[1])
            plt.title('XZ (dir %s): coords: %s' % (direction, coordinates))                        
        elif direction == 2:
            plt.axhline(coordinates[1], c=seaborn.color_palette()[0])
            plt.axvline(coordinates[0], c=seaborn.color_palette()[2])       
            plt.title('Slice %s: coords: %s' % (coordinates[2], coordinates))                                
        # plt.show()
    return(extractedslice)

In [ ]:
# Test extraction script
plt.figure(11)
for direction in range(3):
    plt.subplot(1,3,direction+1)
    # Coordinates are of left blood vessel on slice 2434 on 11um stack of d152, as shown in OrthoViewer of ImageJ
    extract_directional_slice(Reconstructions[15],
                              direction,
                              (1344,2026,2434),
                              verbose=True)
plt.show()

In [ ]:
def markregion(image, coordinates, width, height, showimage=True, center=False, rectangle=True, extract=False):
    """
    Mark a rectancular region in an image.
    For convenience, we can also extract the region if extract is set to true
    """
    from matplotlib.patches import Rectangle
    if len(image.shape) == 2:
        # Nothing to show for stacks
        if showimage:
            plt.imshow(skimage.exposure.equalize_adapthist(image))
        if center:
            plt.scatter(coordinates[0], coordinates[1], color=seaborn.color_palette()[0], marker='x')
            plt.gca().annotate('x=%s, y=%s' % (coordinates[0], coordinates[1]),
                               color='white',
                               xy=(coordinates[0], coordinates[1]),
                               xycoords='data', ha='center', va='top')
        
        if rectangle:
            plt.gca().add_patch(Rectangle((coordinates[0] - width / 2, coordinates[1] - height / 2), width, height,
                                          # edgecolor=seaborn.color_palette()[0],
                                          edgecolor='white',
                                          linestyle='--',
                                          facecolor='none'))
    if extract:
        if len(image.shape) == 3:
            return(image[coordinates[2]][coordinates[0]-width//2:coordinates[0]+width//2, coordinates[1]-height//2:coordinates[1]+height//2])
        elif len(image.shape) == 2:
            return(image[coordinates[0]-width//2:coordinates[0]+width//2, coordinates[1]-height//2:coordinates[1]+height//2])
    else:
        return()

In [ ]:
# Test ROI extraction
a = markregion(Reconstructions[0],
               (1200, 750, 1634),
               width=1250, height=750,
               showimage=True, extract=True).compute()

In [ ]:
plt.imshow(a)
plt.show()

In [ ]:
def extract_roi(stack, coordinates, width=None, height=None, verbose=False):
    """Get a (small) region from an input stack"""
    extract=stack[coordinates[2]][coordinates[1] - int(height/2):coordinates[1] + int(height/2),
                                  coordinates[0] - int(width/2):coordinates[0]+int(width/2)]
    if verbose:
        plt.figure(11)
        plt.subplot(121)
        # plt.imshow(stack[coordinates[2]])
        # show region mark
        markregion(stack[coordinates[2]], coordinates, width, height, center=True)
        plt.title('Slice %s of input stack\n'
                  'Centered at x=%s and y=%s' % (coordinates[2], coordinates[0], coordinates[1]))
        plt.subplot(122)
        plt.imshow(extract)
        plt.title('Extract\n'
                 '%s x %s px' % (extract.shape[0], extract.shape[1]))
        plt.show()
    return(extract)

In [ ]:
# Test ROI extraction
crop = extract_roi(Reconstructions[0],
                   (1200, 750, 1634),
                   width=1250, height=750,
                   verbose=True).compute()

In [ ]:
def extract_roi_2d(image, coordinates, width=None, height=None, verbose=False):
    """Get a (small) region from an input image"""
    if len(coordinates) !=2:
        print('I need 2D coordinates')
        return()
    extract=image[coordinates[1] - int(height/2):coordinates[1] + int(height/2),
    coordinates[0] - int(width/2):coordinates[0]+int(width/2)]
    if verbose:
        plt.figure(11)
        plt.subplot(121)
        markregion(image, coordinates, width, height, center=True)
        plt.title('Centered at x=%s and y=%s' % (coordinates[0], coordinates[1]))
        plt.subplot(122)
        plt.imshow(extract)
        plt.title('Extract\n'
                 '%s x %s px' % (extract.shape[0], extract.shape[1]))
        plt.show()
    return(extract)

In [ ]:
extract_roi_2d(crop, (500, 400), width=250, height=300, verbose=True)

In [ ]:
Data.Grayvalue.unique()

----
# Figure 1

We want to show a figure with the gray value curve along the timeframe we stained and scanned.
In addition some representative slices of a dataset at the start, middle and end of the duration.
And some detailed view of marked regions in these slices.

In [ ]:
# Automatically find the one day closest to the middle of the duration.
# https://stackoverflow.com/a/30112305/323100
middleone = Data.index[(Data['Days passed'] - Data['Days passed'].median()).abs().argsort()[:1]][0]
Data.loc[middleone][['Sample', 'Scan', 'Days passed','Voxelsize']]

In [ ]:
# Put coordinates we want to show for this figure into dataframe
Data['Coordinates'] = ''
Data.at[0, 'Coordinates'] = [800, 900, 1634]
Data.at[middleone, 'Coordinates'] = [800, 900, 1634]
Data.at[len(Data)-1, 'Coordinates'] = [1000, 1500, 2614]

In [ ]:
Data[['Sample', 'Scan', 'Voxelsize', 'Coordinates']]

In [ ]:
# Recalculate voxel size scaling (for showing both 20 um and 11 um scans)
11.0/20.0

In [ ]:
# Extract relevant regions from relevant images
width = 900
height = 900
detail_start = extract_roi(Reconstructions[0],
                           Data['Coordinates'][0],
                           width=width, height=height, verbose=True)
detail_mid = extract_roi(Reconstructions[middleone],
                         Data['Coordinates'][middleone],
                         width=width, height=height, verbose=True)
# Scale width/height with voxel size difference (11/20 um)
detail_end = extract_roi(Reconstructions[-1],
                         Data['Coordinates'].iloc[-1], width=width/.55, height=height/.55, verbose=True)

In [ ]:
Data['Grayvalue'].unique()

In [ ]:
# Convert the scan date to ordinal date, otherwise seaborn cannot plot them with a regplot
# https://stackoverflow.com/a/47877062
from datetime import date
Data['Scan date ordinal'] = pandas.to_datetime(Data['Scan date']).apply(lambda date: date.toordinal())
Data['Time passed ordinal'] = [t - Data['Scan date ordinal'].min() for t in Data['Scan date ordinal']]

In [ ]:
# Plot the mean brightness of the scan with their dates
seaborn.regplot(data=Data,
                x='Scan date ordinal',
                y='MeanBrightness',
                order=2)
plt.ylim(ymin=0)
# Relabel the x-axis with the 'real', not 'ordinal' date
plt.gca().set_xlabel('Scan date')
new_labels = [date.fromordinal(int(item)) for item in plt.gca().get_xticks()]
plt.gca().set_xticklabels(new_labels, rotation=-60)
# Label text: https://matplotlib.org/stable/tutorials/text/annotations.html
for c,row in Data.iterrows():
    plt.gca().annotate('%s/%s' % (row.Sample.replace('Foetus02', 'F2').replace('_Lugol','').replace('_05pct','').replace('_10pct','').replace('_15pct',''), row.Scan),
                       xy=(row['Scan date ordinal'], row.MeanBrightness),
                       xycoords='data',
                       xytext=(-3, -75),
                       textcoords='offset points',
                       ha='left',
                       rotation=-60)
plt.title('Average gray value of the %s-times subsampled reconstructions' % subsample)
plt.show()

In [ ]:
# Plot the *masked* mean brightness of the scan with their dates
seaborn.regplot(data=Data,
                x='Scan date ordinal',
                y='MeanBrightness_masked',
                order=2)
# Relabel the x-axis with the 'real', not 'ordinal' date
plt.gca().set_xlabel('Scan date')
new_labels = [date.fromordinal(int(item)) for item in plt.gca().get_xticks()]
plt.gca().set_xticklabels(new_labels, rotation=-60)
# Label text: https://matplotlib.org/stable/tutorials/text/annotations.html
for c,row in Data.iterrows():
        plt.gca().annotate('%s/%s' % (row.Sample.replace('Foetus02', 'F2').replace('_Lugol','').replace('_05pct','').replace('_10pct','').replace('_15pct',''), row.Scan),
                           xy=(row['Scan date ordinal'], row.MeanBrightness_masked),
                           xycoords='data',
                           xytext=(-3, -75),
                           textcoords='offset points',
                           ha='left',
                           rotation=-60)
plt.ylim(ymin=0)
plt.title('Average gray value of the %s-times subsampled reconstructions (masked > Otsu)' % subsample)
plt.show()

In [ ]:
MeanBrightness_Copy = Data[Data['Grayvalue'] == 0.04]

In [ ]:
# Plot the mean brightness of the scan with their dates
seaborn.regplot(data=MeanBrightness_Copy,
                x='Scan date ordinal',
                y='MeanBrightness',
                order=2
                )
# Relabel the x-axis with the 'real', not 'oridnal date
plt.gca().set_xlabel('Scan date')
new_labels = [date.fromordinal(int(item)) for item in plt.gca().get_xticks()]
plt.gca().set_xticklabels(new_labels, rotation=-60)
# Label text: https://matplotlib.org/stable/tutorials/text/annotations.html
for c,row in MeanBrightness_Copy.iterrows():
        plt.gca().annotate('%s/%s' % (row.Sample.replace('Foetus02', 'F2').replace('_Lugol','').replace('_05pct','').replace('_10pct','').replace('_15pct',''), row.Scan),
                       xy=(row['Scan date ordinal'], row.MeanBrightness),
                       xycoords='data',
                       xytext=(-3, -75),
                       textcoords='offset points',
                       ha='left',
                       rotation=-60)
plt.ylim(ymin=0)
plt.title('Average gray value of the %s-times subsampled reconstructions (CS GV=0.04)' % subsample)
plt.savefig('Helligkeit.png')
plt.show()

In [ ]:
# Plot the mean brightness of the scan with their dates
seaborn.regplot(data=MeanBrightness_Copy,
                x='Time passed ordinal',
                y='MeanBrightness_masked',
                order=2
                )
# Relabel the x-axis with the 'real', not 'oridnal date
plt.gca().set_xlabel('Days since start')
# new_labels = [date.fromordinal(int(item)) for item in plt.gca().get_xticks()]
# plt.gca().set_xticklabels(new_labels, rotation=-60)
# Label text: https://matplotlib.org/stable/tutorials/text/annotations.html
for c,row in MeanBrightness_Copy.iterrows():
            plt.gca().annotate('%s/%s' % (row.Sample.replace('Foetus02', 'F2').replace('_Lugol','').replace('_05pct','').replace('_10pct','').replace('_15pct',''), row.Scan),
                       xy=(row['Scan date ordinal'], row.MeanBrightness_masked),
                       xycoords='data',
                       xytext=(-3, -75),
                       textcoords='offset points',
                       ha='left',
                       rotation=-60)
plt.ylim(ymin=0)
plt.title('Average gray value of the %s-times subsampled reconstructions (CS GV=0.04) (masked > Otsu)' % subsample)
plt.savefig('Helligkeit.png')
plt.show()

In [ ]:
plt.rcParams['figure.figsize']

In [ ]:
# Generate figure 1
# plt.rcParams['figure.figsize'][1] = figsize_default[0]
fig = plt.figure(1)
gs = GridSpec(3, 3, figure=fig)
fig.add_subplot(gs[0, :])
# Plot the mean brightness of the scan with their dates
seaborn.regplot(data=MeanBrightness_Copy,  # Only show 'low resolution scans', others should *not* be matched. We can easily do that with excluding all the ones that have not used 0.04 as max grayvalue in the reconstructions
               x='Time passed ordinal',  # Plot the gray values with 'time passed' instead of scan date
               y='MeanBrightness_masked',
               order=2)
# Annotate the the samples we use below
from string import ascii_uppercase    
for c, i in enumerate([0, middleone, len(Data)-2]): # Use len(Data)-*2* here, because otherwise we label at the y-value of the not-shown HR scan, which looks not so good.
    plt.gca().annotate('%s/%s' % (ascii_uppercase[c+1], ascii_uppercase[c+1+3]),
                       xy = (Data['Time passed ordinal'][i], Data['MeanBrightness_masked'][i] - 5),
                       xycoords='data', ha='center', va='top')
plt.xlabel('Days passed')
plt.ylabel('Average gray value')
plt.ylim(ymin=0)
if subsample != 1:
    plt.title('Average gray value in the sample, calculated from %s-times subsampled reconstructions' % subsample)    
seaborn.despine()
# Add overview images
fig.add_subplot(gs[1, 0])
markregion(Reconstructions[0][Data['Coordinates'][0][2]], 
           Data['Coordinates'][0], width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][0],'um'))
label_image(Reconstructions[0][Data['Coordinates'][0][2]], 'B')
plt.axis('off')
fig.add_subplot(gs[1, 1])
markregion(Reconstructions[middleone][Data['Coordinates'][middleone][2]],
           Data['Coordinates'][middleone], width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][middleone],'um'))
label_image(Reconstructions[middleone][Data['Coordinates'][middleone][2]], 'C')
plt.axis('off')
fig.add_subplot(gs[1, 2])
markregion(Reconstructions[-1][Data['Coordinates'].iloc[-1][2]],
           Data['Coordinates'].iloc[-1],
           width=width/.55, height=height/.55)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(Reconstructions[-1][Data['Coordinates'].iloc[-1][2]], 'D')
plt.axis('off')
# Add regions
fig.add_subplot(gs[2, 0])
plt.imshow(skimage.exposure.equalize_adapthist(detail_start))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][0],'um'))
label_image(detail_start, 'E')
plt.axis('off')
fig.add_subplot(gs[2, 1])
plt.imshow(skimage.exposure.equalize_adapthist(detail_mid))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][middleone],'um'))
label_image(detail_mid, 'F')
plt.axis('off')
fig.add_subplot(gs[2, 2])
plt.imshow(skimage.exposure.equalize_adapthist(detail_end))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(detail_end, 'G')
plt.axis('off')

plt.tight_layout(pad=0.1)#, h_pad=0.01, w_pad=0.01)

plt.savefig(os.path.join(OutPutDir, 'Fig01.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.png'),
            bbox_inches='tight')
plt.show()
plt.rcParams['figure.figsize'] = figsize_default

In [ ]:
# Generate subdirectory for panels
for i in range(1,4):
    os.makedirs(os.path.join(OutPutDir, 'Fig0%s.panels' % i), exist_ok=True)
    os.makedirs('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures/Fig0%s.panels' % i, exist_ok=True)

In [ ]:
# Save out single panels of figure 1
seaborn.regplot(data=MeanBrightness_Copy,  # Only show 'low resolution scans', others should *not* be matched. We can easily do that with excluding all the ones that have not used 0.04 as max grayvalue in the reconstructions
               x='Time passed ordinal',  # Plot the gray values with 'time passed' instead of scan date
               y='MeanBrightness_masked',
               order=2)
for c, i in enumerate([0, middleone, len(Data)-2]): # Use len(Data)-*2* here, because otherwise we label at the y-value of the not-shown HR scan, which looks not so good.
    plt.gca().annotate('%s/%s' % (ascii_uppercase[c+1], ascii_uppercase[c+1+3]),
                       xy = (Data['Time passed ordinal'][i], Data['MeanBrightness_masked'][i] - 5),
                       xycoords='data', ha='center', va='top')
plt.xlabel('Days passed')
plt.ylabel('Average gray value')
plt.ylim(ymin=0)
if subsample != 1:
    plt.title('Average gray value in the sample, calculated from %s-times subsampled reconstructions' % subsample)    
seaborn.despine()
plt.savefig(os.path.join(OutPutDir, 'Fig01.panels', 'Fig01.A.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.panels', 'Fig01.A.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(Reconstructions[0][Data['Coordinates'][0][2]], 
           Data['Coordinates'][0], width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][0],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig01.panels', 'Fig01.B.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.panels', 'Fig01.B.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(Reconstructions[middleone][Data['Coordinates'][middleone][2]],
           Data['Coordinates'][middleone], width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][middleone],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig01.panels', 'Fig01.C.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.panels', 'Fig01.C.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(Reconstructions[-1][Data['Coordinates'].iloc[-1][2]],
           Data['Coordinates'].iloc[-1],
           width=width/.55, height=height/.55)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig01.panels', 'Fig01.D.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.panels', 'Fig01.D.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(detail_start))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][0],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig01.panels', 'Fig01.E.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.panels', 'Fig01.E.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(detail_mid))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'][middleone],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig01.panels', 'Fig01.F.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.panels', 'Fig01.F.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(detail_end))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(detail_end, 'G')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig01.panels', 'Fig01.G.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig01.panels', 'Fig01.G.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
OutPutDir

----
# Figure 2

Koordinaten (in 11um stack 152d)

A) x: 2249, y: 1075, z:819 --> war gedreht, jetzt 1936,2218,819

B) x:2223 y: 1308 z:2527 --> war gedreht, jetzt 2230,2093,2527

C) x: 1939, y: 1063, z: 819 --> war gedreht, jetzt 2061,1878,819

In [ ]:
# Collect figure 2
plt.figure(2)
plt.subplot(231)
markregion(Reconstructions[-1][819], 
           (1936, 2218, 819), 500, 500)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(Reconstructions[-1][819], 'A')
plt.axis('off')
plt.subplot(232)
markregion(Reconstructions[-1][2527], 
           (2048, 2180, 2527), 700, 700)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(Reconstructions[-1][819], 'B')
plt.axis('off')
plt.subplot(233)
markregion(Reconstructions[-1][819], 
           (2061, 1878, 819), 500, 500)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(Reconstructions[-1][819], 'C')
plt.axis('off')
#---
plt.subplot(234)
plt.imshow(skimage.exposure.equalize_adapthist(extract_roi(Reconstructions[-1],
                                                           (1936, 2218, 819), 500, 500)))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(extract_roi(Reconstructions[-1],
                        (1936, 2218, 819), 500, 500),
            'D')
plt.axis('off')
plt.subplot(235)
plt.imshow(skimage.exposure.equalize_adapthist(extract_roi(Reconstructions[-1],
                                                           (2048, 2180, 2527), 700, 700)))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(extract_roi(Reconstructions[-1],
                        (2048, 2180, 2527), 700, 700),
            'E')
plt.axis('off')
plt.subplot(236)
plt.imshow(skimage.exposure.equalize_adapthist(extract_roi(Reconstructions[-1],
                                                           (2061, 1878, 819), 500, 500)))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(extract_roi(Reconstructions[-1],
                        (2061, 1878, 819), 500, 500),
            'F')
plt.axis('off')
plt.tight_layout(pad=0.3, h_pad=0.3, w_pad=0.3)
plt.savefig(os.path.join(OutPutDir, 'Fig02.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig02.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(Reconstructions[-1][819], 
           (1936, 2218, 819), 500, 500)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig02.panels', 'Fig02.A.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig02.panels', 'Fig02.A.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(Reconstructions[-1][2527], 
           (2048, 2180, 2527), 700, 700)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig02.panels', 'Fig02.B.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig02.panels', 'Fig02.B.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(Reconstructions[-1][819], 
           (2061, 1878, 819), 500, 500)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig02.panels', 'Fig02.C.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig02.panels', 'Fig02.C.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(extract_roi(Reconstructions[-1],
                                                           (1936, 2218, 819), 500, 500)))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig02.panels', 'Fig02.D.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig02.panels', 'Fig02.D.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(extract_roi(Reconstructions[-1],
                                                           (2048, 2180, 2527), 700, 700)))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig02.panels', 'Fig02.E.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig02.panels', 'Fig02.E.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(extract_roi(Reconstructions[-1],
                                                           (2061, 1878, 819), 500, 500)))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig02.panels', 'Fig02.F.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig02.panels', 'Fig02.F.png'),
            bbox_inches='tight')
plt.show()

----
# Figure 3

Koordinaten: neu, im rotierten 11um stack 152 days 

Bulbospongiosus (+ ischiocavernosus) 

- YZ 1935, x 723, y 842 --> 1935,723,842
- XZ 2229, x 1935, y 722 -> 1935,2229,722

Longitudinal rectum muscle 

- XY (original orientation) slice 789, x 2022, y 1932 
- YZ 2022, x 789, y 1139 

EAS 

- YZ 1752, x 489, y 1442 
- XZ 1629, x 1752, y 488 

STP 
- Slice 651, x 2010, y 1914, z 650 
- XZ 1914, x 2010, y 650 

In [ ]:
# Which is which?
# Copy from function-definition above
    # if direction == 0:
    #     plt.title('YZ (dir %s): coords: %s' % (direction, coordinates))            
    # elif direction == 1:
    #     plt.title('XZ (dir %s): coords: %s' % (direction, coordinates))                        
    # elif direction == 2:
    #     plt.title('Slice %s: coords: %s' % (coordinates[2], coordinates))                                

In [ ]:
coordinates_bsp_a = (1935,723,842)
coordinates_bsp_b = (1935,2229,722)
coordinates_lrm_a = (2022,1932,789)
coordinates_lrm_b = (2022,1932,789)
coordinates_eas_a = (1752,1442,489)
coordinates_eas_b = (1752,1629,488)
coordinates_stp_a = (2010,1914,650)
coordinates_stp_b = (2010,1914,650)

In [ ]:
width=500
height=600

In [ ]:
# Bulbospongiosus
bsp_a = extract_directional_slice(Reconstructions[15],
                                  0,
                                  coordinates_bsp_a,
                                  verbose=False).compute()

In [ ]:
plt.subplot(121)
markregion(bsp_a, coordinates_bsp_a, width, height)
bsp_a_d = extract_roi_2d(bsp_a, (coordinates_bsp_a[0], coordinates_bsp_a[1]), width, height)
plt.subplot(122)
plt.imshow(bsp_a_d)
plt.show()

In [ ]:
# Bulbospongiosus
bsp_b = extract_directional_slice(Reconstructions[15],
                                  1,
                                  coordinates_bsp_b,
                                  verbose=False).compute()

In [ ]:
coordinates_bsp_b
plt.subplot(121)
markregion(bsp_b, coordinates_bsp_b, width, height)
bsp_b_d = extract_roi_2d(bsp_b, (coordinates_bsp_b[0], coordinates_bsp_b[1]), width, height)
plt.subplot(122)
plt.imshow(bsp_b_d)
plt.show()

In [ ]:
# Longitudinal rectum muscle 
lrm_a = extract_directional_slice(Reconstructions[15],
                                  2,
                                  coordinates_lrm_a,
                                  verbose=False).compute()

In [ ]:
plt.subplot(121)
markregion(lrm_a, coordinates_lrm_a, width, height)
lrm_a_d = extract_roi_2d(lrm_a, (coordinates_lrm_a[0], coordinates_lrm_a[1]), width, height)
plt.subplot(122)
plt.imshow(lrm_a_d)
plt.show()

In [ ]:
# Longitudinal rectum muscle 
lrm_b = extract_directional_slice(Reconstructions[15],
                                  1,
                                  coordinates_lrm_b,
                                  verbose=False).compute()
lrm_b_d = extract_roi_2d(lrm_b, (coordinates_lrm_b[0], coordinates_lrm_b[1]), width, height)

In [ ]:
# EAS
eas_a = extract_directional_slice(Reconstructions[15],
                                  0,
                                  coordinates_eas_a,
                                  verbose=False).compute()
eas_a_d = extract_roi_2d(eas_a, (coordinates_eas_a[0], coordinates_eas_a[1]), width, height)

In [ ]:
# EAS
eas_b = extract_directional_slice(Reconstructions[15],
                                  1,
                                  coordinates_eas_b,
                                  verbose=False).compute()
eas_b_d = extract_roi_2d(eas_b, (coordinates_eas_b[0], coordinates_eas_b[1]), width, height)

In [ ]:
# STP
stp_a = extract_directional_slice(Reconstructions[15],
                                  2,
                                  (2010,1914,650),
                                  verbose=False).compute()
stp_a_d = extract_roi_2d(stp_a, (coordinates_stp_a[0], coordinates_stp_a[1]), width, height)

In [ ]:
# STP
stp_b = extract_directional_slice(Reconstructions[15],
                                  1,
                                  coordinates_stp_b,
                                  verbose=False).compute()
stp_b_d = extract_roi_2d(stp_b, (coordinates_stp_b[0], coordinates_stp_b[1]), width, height)

In [ ]:
# Collect figure 3
width = 500
height = 600
plt.rcParams['figure.figsize'][1] = figsize_default[0] * 1.1
plt.figure(3)
plt.subplot(4,4,1)
markregion(bsp_a, coordinates_bsp_a, width, height)
label_image(bsp_a,'A')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.figure(3)
plt.subplot(4,4,2)
plt.imshow(skimage.exposure.equalize_adapthist(bsp_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(bsp_a_d,'B')
plt.axis('off')
plt.subplot(4,4,3)
markregion(bsp_b, coordinates_bsp_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(bsp_b,'C')
plt.axis('off')
plt.subplot(4,4,4)
plt.imshow(skimage.exposure.equalize_adapthist(bsp_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(bsp_b_d,'D')
plt.axis('off')
# ----------
plt.subplot(4,4,5)
markregion(lrm_a, coordinates_lrm_a, width, height)
label_image(lrm_a,'E')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.figure(3)
plt.subplot(4,4,6)
plt.imshow(skimage.exposure.equalize_adapthist(lrm_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(lrm_a_d,'F')
plt.axis('off')
plt.subplot(4,4,7)
markregion(lrm_b, coordinates_lrm_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(lrm_b,'G')
plt.axis('off')
plt.subplot(4,4,8)
plt.imshow(skimage.exposure.equalize_adapthist(lrm_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(eas_b_d,'H')
plt.axis('off')
# # ----------
plt.subplot(4,4,9)
markregion(eas_a, coordinates_eas_a, width, height)
label_image(eas_a,'I')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.figure(3)
plt.subplot(4,4,10)
plt.imshow(skimage.exposure.equalize_adapthist(eas_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(eas_a_d,'J')
plt.axis('off')
plt.subplot(4,4,11)
markregion(eas_b, coordinates_eas_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(eas_b,'K')
plt.axis('off')
plt.subplot(4,4,12)
plt.imshow(skimage.exposure.equalize_adapthist(eas_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(lrm_b_d,'L')
plt.axis('off')
# # ----------
plt.subplot(4,4,13)
markregion(stp_a, coordinates_stp_a, width, height)
label_image(stp_a,'M')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.figure(3)
plt.subplot(4,4,14)
plt.imshow(skimage.exposure.equalize_adapthist(stp_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(stp_a_d,'N')
plt.axis('off')
plt.subplot(4,4,15)
markregion(stp_b, coordinates_stp_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(stp_b,'O')
plt.axis('off')
plt.subplot(4,4,16)
plt.imshow(skimage.exposure.equalize_adapthist(stp_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
label_image(stp_b_d,'P')
plt.axis('off')
plt.tight_layout(pad=0.3, h_pad=0.3, w_pad=0.3)
plt.savefig(os.path.join(OutPutDir, 'Fig03.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.png'),
            bbox_inches='tight')
plt.show()
plt.rcParams['figure.figsize'] = figsize_default

In [ ]:
# Save single panels
markregion(bsp_a, coordinates_bsp_a, width, height)
# label_image(bsp_a,'A')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.A.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.A.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(bsp_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(bsp_a_d,'B')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.B.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.B.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(bsp_b, coordinates_bsp_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(bsp_b,'C')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.C.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.C.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(bsp_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(bsp_b_d,'D')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.D.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.D.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(lrm_a, coordinates_lrm_a, width, height)
# label_image(lrm_a,'E')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.E.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.E.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(lrm_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(lrm_a_d,'F')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.F.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.F.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(lrm_b, coordinates_lrm_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(lrm_b,'G')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.G.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.G.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(lrm_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(eas_b_d,'H')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.H.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.H.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(eas_a, coordinates_eas_a, width, height)
# label_image(eas_a,'I')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.I.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.I.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(eas_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(eas_a_d,'J')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.J.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.J.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(eas_b, coordinates_eas_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(eas_b,'K')
plt.axis('off')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.K.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.K.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(eas_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(lrm_b_d,'L')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.L.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.L.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(stp_a, coordinates_stp_a, width, height)
# label_image(stp_a,'M')
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.M.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.M.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(stp_a_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(stp_a_d,'N')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.N.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.N.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
markregion(stp_b, coordinates_stp_b, width, height)
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(stp_b,'O')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.O.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.O.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
plt.imshow(skimage.exposure.equalize_adapthist(stp_b_d))
plt.gca().add_artist(ScaleBar(Data['Voxelsize'].iloc[-1],'um'))
# label_image(stp_b_d,'P')
plt.axis('off')
plt.savefig(os.path.join(OutPutDir, 'Fig03.panels', 'Fig03.P.png'),
            bbox_inches='tight')
plt.savefig(os.path.join('/home/habi/research-storage-djonov/Aaldijk/PelvicFloor/Pelvis-Manuscript/Figures', 'Fig03.panels', 'Fig03.P.png'),
            bbox_inches='tight')
plt.show()

In [ ]:
print('Saved all figures to %s' % OutPutDir)